# Assignment 9
## Applied Machine Learning

Andrew Chan 
EBE869

# 1. [10 pts] Describe the environment in the Nim learning model.

The environment is a game that provides the following:
* 3 piles of cards
* For each pile, a maximum of 10 cards per pile

Thus, the states are 
* the number of items in the 1st pile
* the number of items in the 2nd pile
* the number of items in the 3rd pile

# 2. [10 pts] Describe the agent in the Nim learning model.

The agent is a player that can perform the following: 
* select a pile 
* remove at least 1 card from the pile

Thus, the actions are 
* number of items to remove
* pile ID

# 3. [10 pts] Describe the reward and penalty in the Nim learning model.

The reward is only given upon winning the game.

# 4. [10 pts] How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?


Since we can have 0 cards per pile the total number of states = $${(ITEMS\_MX + 1)}^{3}$$ 

In [12]:
ITEMS_MX = 10

In [13]:
(ITEMS_MX+1)**3

1331

Thus, `1331` possible states.

# 5. [10 pts] How many possible actions there could be in the Nim game with 10 items per pile and 3 piles total?


Since we must remove at least 1 card per pile the total number of actions = $${ITEMS\_MAX}\times{3}$$

In [14]:
(ITEMS_MX)*3

30

Thus, `30` possible actions.

# 6. [50 pts] Find a way to improve the provided Nim game learning model. 
Do you think one can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed possible to find a better solution, which improves the way Q-learning updates its Q-table)

In [15]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [16]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action

In [17]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")
    #
    return wins['A'], wins['B']

In [18]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random  501    Random  499
1000 games,     Guru  999    Random    1
1000 games,   Random    9      Guru  991
1000 games,     Guru  928      Guru   72


In [19]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
            #
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [20]:
%%time
nim_qlearn(100000)

Wall time: 16.5 s


In [25]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner') ;
play_games(1000, 'Guru', 'Qlearner') ;
play_games(1000, 'Qlearner', 'Guru') ;

1000 games, Qlearner  708    Random  292
1000 games,   Random  265  Qlearner  735
1000 games,     Guru  997  Qlearner    3
1000 games, Qlearner   24      Guru  976


In [22]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner  542    Random  458
1000 games, Qlearner  554    Random  446
1000 games, Qlearner  677    Random  323
1000 games, Qlearner  730    Random  270
1000 games, Qlearner  733    Random  267
1000 games, Qlearner  709    Random  291
1000 games, Qlearner  702    Random  298
Wall time: 27.7 s


In [23]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(wins)

[0.542, 0.554, 0.677, 0.73, 0.733, 0.709, 0.702]


In [24]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        #
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            #
            print(s, file=fout)
#
qtable_log('qtable_debug_module09.txt')